In [10]:
!pip install llama-index python-dotenv

In [2]:
from llama_index.core.query_pipeline import QueryPipeline

chinook database をサンプルデータとして利用する。

In [3]:
%pip install llama-index-llms-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
!curl "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O ./chinook.zip
!unzip -o ./chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0   419k      0 --:--:-- --:--:-- --:--:--  420k
curl: (6) Could not resolve host: .
Archive:  ./chinook.zip
  inflating: chinook.db              


In [5]:
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")
sql_database = SQLDatabase(engine)

global callback manager をセットアップする。

In [6]:
from llama_index.core.settings import Settings
from llama_index.core.callbacks import CallbackManager

callback_manager = CallbackManager()
Settings.callback_manager = callback_manager

In [11]:
from dotenv import load_dotenv

load_dotenv()

True

setup a simple text-to-SQL tool: given a query, translate text to SQL, execute against database, and get back a result.

In [12]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True
)

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    )
)

次に、クエリ パイプライン構文を使用して、単一ステップの ReAct パイプラインをセットアップします。 これは、次のことを行う複数の部分からなるプロセスです。

1. エージェントの入力を取り込む
2. LLM を使用して ReAct プロンプトを呼び出し、次のアクション/ツールを生成します (または応答を返します)。
3. ツール/アクションが選択されている場合は、ツール パイプラインを呼び出してツールを実行し、応答を収集します。
4. レスポンスが発生した場合はレスポンスを取得します。

この全体を通じて、エージェント固有のさまざまなクエリ コンポーネントを使用します。 通常のクエリ パイプラインとは異なり、これらは QueryPipelineAgentWorker で使用されるクエリ パイプライン用に特別に設計されています。

- AgentInputComponent は、エージェント入力 (タスク、状態ディクショナリ) をクエリ パイプラインの入力セットに変換できるようにします。
- AgentFnComponent: 現在のタスク、状態、および任意の入力を取り込み、出力を返すことができる汎用プロセッサです。 このクックブックでは、ReAct プロンプトをフォーマットする関数コンポーネントを定義します。 ただし、これはどこにでも置くことができます。
- [このノートブックでは使用されません] CustomAgentComponent: AgentFnComponent と同様に、_run_component を実装して、タスクと状態にアクセスできる独自のロジックを定義できます。 AgentFnComponent よりも冗長ですが、より柔軟です (たとえば、初期化変数を定義でき、コールバックは基本クラス内にあります)。
AgentFnComponent および AgentInputComponent に渡される関数には、タスクと状態がエージェントから渡される入力であるため、入力変数としてタスクと状態を含める必要があることに注意してください。

エージェント クエリ パイプラインの出力は Tuple[AgentChatResponse, bool] でなければならないことに注意してください。 これは以下でわかります。

In [13]:
from llama_index.core.query_pipeline import QueryPipeline as QP

qp = QP(verbose=True)

ここでは、すべてのエージェント ステップの開始時に呼び出されるエージェント入力コンポーネントを定義します。 入力を渡すだけでなく、初期化/状態変更も行います。

In [14]:
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep
)
from llama_index.core.agent import Task, AgentChatResponse
from llama_index.core.query_pipeline import (
    AgentInputComponent,
    AgentFnComponent,
    CustomAgentComponent,
    QueryComponent,
    ToolRunnerComponent
)
from llama_index.core.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast


## Agent Input Component
## This is the component that produces agent inputs to the rest of the components can also put initialization logic here.
def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict[str, Any]:
    """Agent input function.
    
    Returns:
        A Dictionary of output keys and values. If you are specifying src_key when defining links between this component and other components, make sure the src_key matches the specified output_key.

    """
    # initialize current_reasoning
    if "current_reasoning" not in state:
        state["current_reasoning"] = []
    reasoning_step = ObservationReasoningStep(observation=task.input)
    state["current_reasoning"].qppend(reasoning_step)
    return {"input": task.input}


agent_input_component = AgentInputComponent(fn=agent_input_fn)

ここでは、ReAct プロンプトを生成するエージェント コンポーネントを定義し、LLM から出力が生成された後、構造化オブジェクトに解析します。

In [15]:
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.query_pipeline import InputComponent, Link
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool


## define prompt function
def react_prompt_fn(
    task: Task, state: Dict[str, Any], input: str, tools: List[BaseTool]
) -> List[ChatMessage]:
    # Add input to reasoning
    chat_formatter = ReActChatFormatter()
    return chat_formatter.format(
        tools,
        chat_history=task.memory.get() + state["memory"].get_all(),
        current_reasoning=state["current_reasoning"]
    )

react_prompt_component = AgentFnComponent(
    fn=react_prompt_fn, partial_dict={"tools": [sql_tool]}
)

In [17]:
from llama_index.core.base.llms.generic_utils import messages_to_prompt

chat_formatter = ReActChatFormatter()
msgs = chat_formatter.format(
    [sql_tool],
    chat_history=[],
    current_reasoning=[]
)
print(messages_to_prompt(msgs))

system: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
> Tool Name: sql_tool
Tool Description: Useful for translating a natural language query into a SQL query
Tool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}



## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of sql_tool) if using a tool.
Action Input: the input to the tool, in a JSON format represent